In [2]:
import pandas as pd
import json
from sqlalchemy import create_engine
from haversine import haversine

In [3]:
#READ JSON CONTENT
with open('../dataset/starlink_historical_data.json') as f:
    data = json.loads(f.read())
df = pd.json_normalize(data) 

In [4]:
df = df[['id','spaceTrack.CREATION_DATE', 'longitude', 'latitude']]
df.rename(columns = {'spaceTrack.CREATION_DATE':'creation_date'}, inplace = True)

In [5]:
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')

In [6]:
df.to_sql('starlink_historical_data', engine)

In [7]:
df_pg = pd.read_sql_query('Select * from starlink_historical_data', con=engine)
df_pg.head()

,index,id,creation_date,longitude,latitude
0,0,5eed770f096e59000698560d,2020-10-13T04:16:08,86,NaN
1,1,5eed770f096e59000698560e,2020-09-28T19:26:08,48,NaN
2,2,5eed770f096e59000698560f,2020-10-13T17:46:09,62,NaN
3,3,5eed770f096e590006985610,2021-01-26T02:30:00,109,25.453949
4,4,5eed770f096e590006985611,2020-09-02T18:57:38,118,NaN


In [8]:
#PART 3
#input parameter time to search
time_to_search = '2022-01-01'

query_last_position = """
select * 
from starlink_historical_data as s
inner join 
  (select id, max(creation_date) as creation_date
  from starlink_historical_data
  where creation_date <= '%s'
  group by id) as pivot 
on s.id = pivot.id and s.creation_date = pivot.creation_date
order by s.id
"""%time_to_search

df_pg = pd.read_sql_query(query_last_position, con=engine)
df_pg.head()

,index,id,creation_date,longitude,latitude,id,creation_date
0,0,5eed770f096e59000698560d,2020-10-13T04:16:08,86,NaN,5eed770f096e59000698560d,2020-10-13T04:16:08
1,1,5eed770f096e59000698560e,2020-09-28T19:26:08,48,NaN,5eed770f096e59000698560e,2020-09-28T19:26:08
2,2,5eed770f096e59000698560f,2020-10-13T17:46:09,62,NaN,5eed770f096e59000698560f,2020-10-13T17:46:09
3,3,5eed770f096e590006985610,2021-01-26T02:30:00,109,25.453949,5eed770f096e590006985610,2021-01-26T02:30:00
4,4,5eed770f096e590006985611,2020-09-02T18:57:38,118,NaN,5eed770f096e590006985611,2020-09-02T18:57:38


In [9]:
def calculate_haversine(row, input_coordinate):
    result = abs(haversine((row['latitude'], row['longitude']), input_coordinate))
    return pd.Series([result])

In [10]:
#PART 4
#input parameter time to search
time_to_search = '2022-01-01'
#input parameter time to search
longitude_to_search = 150.0
#input parameter time to search
latitude_to_search = 24.5

input_coordinate = (latitude_to_search, longitude_to_search)

query_last_position = """
select * 
from starlink_historical_data as s
inner join 
  (select id, max(creation_date) as creation_date
  from starlink_historical_data
  where creation_date <= '%s'
  group by id) as pivot 
on s.id = pivot.id and s.creation_date = pivot.creation_date
order by s.id
"""%time_to_search

df_pg = pd.read_sql_query(query_last_position, con=engine)
df_pg.head()

df_pg['haversine_result'] = df_pg.apply(lambda row: calculate_haversine(row, input_coordinate), axis=1)

closest_one = df_pg.iloc[df_pg.haversine_result.idxmin()]

'closest satellite id %s'%closest_one.id



'closest satellite id id    5f5a9c1d2fd30c00065e5e64\nid    5f5a9c1d2fd30c00065e5e64\nName: 767, dtype: object'